In [ ]:
import sys
import os

# Add the project directory to the path
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_dir = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(project_dir)

from src. ollama_utils import ollama_quick_chat

import json
import time

In [ ]:
lecture_transcripts = {}

for txt_file in ["TL1 - Full", "TL2 - Full", "TL3 - Full", "TL4 - Full"]:
    with open(f"../data/{txt_file}.txt", "r") as file:
        lecture_transcripts[txt_file] = file.read()

In [ ]:
with open("../data/prompts.json", "r") as f:
    prompts = json.load(f)

task = prompts["tasks"][0]

user_prompt = task["user_prompt"].format(transcript="This is the transcript content.")
print(user_prompt)

In [ ]:
output = {}

for transcript_name, transcript in lecture_transcripts.items():

    print("Working on: ", transcript_name, "...")

    tmp = {}
    
    for task in range(len(prompts["tasks"])):

        print("        ...conducting task: ", task)

        if task in [0, 1, 2, 4]:

            system_prompt = prompts["tasks"][task]["system_prompt"]
            user_prompt = prompts["tasks"][task]["user_prompt"].format(transcript=transcript)

            tmp[task] = ollama_quick_chat(system_message=system_prompt, user_input=user_prompt, llm="llama3.3:70b")["message"]["content"]

        elif task == 3:

            system_prompt = prompts["tasks"][task]["system_prompt"]
            user_prompt = prompts["tasks"][task]["user_prompt"].format(transcript=transcript, concepts=tmp[2])

            tmp[task] = ollama_quick_chat(system_message=system_prompt, user_input=user_prompt, llm="llama3.3:70b")["message"]["content"]

        elif task == 5:

            system_prompt = prompts["tasks"][task]["system_prompt"]
            user_prompt = prompts["tasks"][task]["user_prompt"].format(transcript=transcript, questions=tmp[4])

            tmp[task] = ollama_quick_chat(system_message=system_prompt, user_input=user_prompt, llm="llama3.3:70b")["message"]["content"]

        elif task == 6:

            system_prompt = prompts["tasks"][task]["system_prompt"]
            user_prompt = prompts["tasks"][task]["user_prompt"].format(transcript=transcript, questions=tmp[4], answers=tmp[5])

            tmp[task] = ollama_quick_chat(system_message=system_prompt, user_input=user_prompt, llm="llama3.3:70b")["message"]["content"]

        time.sleep(30)
    time.sleep(120)

    output[transcript_name] = tmp

In [ ]:
with open("../data/output.json", "w") as file:
    json.dump(output, file, indent=4)